In [1]:
import torch, sys, os
sys.path.append("../../modules/imagenet")
import datapipe as dp
import dit, time, warnings
import imagenet_maps as im
from diffusers.models import AutoencoderKL
warnings.filterwarnings("ignore")

root = "../../data/ImageNet-1k/2012"
data_dir = "../../data/tiny-imagenet-200/train"
device = "mps"
JSON = "../../data/ImageNet-1k/imagenet_class_index.json"
model, vae = dit.load_DiT("../../data/ImageNet-1k/DiT-XL-2", device)
diffusion =  dit.load_diffusion(15)

In [6]:
start = time.time()
imgs = dit.generate_uncond_batched_optimized(model, vae, diffusion, 100, device, batch_size=1)
print("Time taken {}".format(time.time() - start))

Time taken 243.5180208683014


In [3]:
imgs.shape

torch.Size([100, 3, 256, 256])